In [ ]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque

from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Flatten


# 상태가 입력, 큐함수가 출력인 인공신경망 생성
class DQN(tf.keras.Model):
    def __init__(self, action_size, state_size):
        super(DQN, self).__init__()
        self.conv1 = Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                            input_shape=state_size)
        self.conv2 = Conv2D(64, (4, 4), strides=(2, 2), activation='relu')
        self.conv3 = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')
        self.flatten = Flatten()
        self.fc = Dense(512, activation='relu')
        self.fc_out = Dense(action_size)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc(x)
        q = self.fc_out(x)
        return q


# 브레이크아웃 예제에서의 DQN 에이전트
class DQNAgent:
    def __init__(self, action_size, state_size=(84, 84, 4)):
        self.render = True

        # 상태와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        # DQN 하이퍼파라미터
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.02
        self.exploration_steps = 1000000.
        self.epsilon_decay_step = self.epsilon_start - self.epsilon_end
        self.epsilon_decay_step /= self.exploration_steps
        self.batch_size = 32
        self.train_start = 50000
        self.update_target_rate = 10000

        # 리플레이 메모리, 최대 크기 100,000
        self.memory = deque(maxlen=100000)
        # 게임 시작 후 랜덤하게 움직이지 않는 것에 대한 옵션
        self.no_op_steps = 30

        # 모델과 타깃 모델 생성
        self.model = DQN(action_size, state_size)
        self.target_model = DQN(action_size, state_size)
        self.optimizer = Adam(self.learning_rate, clipnorm=10.)
        # 타깃 모델 초기화
        self.update_target_model()

        self.avg_q_max, self.avg_loss = 0, 0

        self.writer = tf.summary.create_file_writer('summary/breakout_dqn')
        self.model_path = os.path.join(os.getcwd(), 'save_model', 'model')

    # 타깃 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model(history)
            return np.argmax(q_value[0])

    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, history, action, reward, next_history, dead):
        self.memory.append((history, action, reward, next_history, dead))

    # 텐서보드에 학습 정보를 기록
    def draw_tensorboard(self, score, step, episode):
        with self.writer.as_default():
            tf.summary.scalar('Total Reward/Episode', score, step=episode)
            tf.summary.scalar('Average Max Q/Episode',
                              self.avg_q_max / float(step), step=episode)
            tf.summary.scalar('Duration/Episode', step, step=episode)
            tf.summary.scalar('Average Loss/Episode',
                              self.avg_loss / float(step), step=episode)

    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        if self.epsilon > self.epsilon_end:
            self.epsilon -= self.epsilon_decay_step

        # 메모리에서 배치 크기만큼 무작위로 샘플 추출
        batch = random.sample(self.memory, self.batch_size)

        history = np.array([sample[0][0] / 255. for sample in batch],
                           dtype=np.float32)
        actions = np.array([sample[1] for sample in batch])
        rewards = np.array([sample[2] for sample in batch])
        next_history = np.array([sample[3][0] / 255. for sample in batch],
                                dtype=np.float32)
        dones = np.array([sample[4] for sample in batch])

        # 학습 파라메터
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            # 현재 상태에 대한 모델의 큐함수
            predicts = self.model(history)
            one_hot_action = tf.one_hot(actions, self.action_size)
            predicts = tf.reduce_sum(one_hot_action * predicts, axis=1)

            # 다음 상태에 대한 타깃 모델의 큐함수
            target_predicts = self.target_model(next_history)

            # 벨만 최적 방정식을 구성하기 위한 타깃과 큐함수의 최대 값 계산
            max_q = np.amax(target_predicts, axis=1)
            targets = rewards + (1 - dones) * self.discount_factor * max_q

            # 후버로스 계산
            error = tf.abs(targets - predicts)
            quadratic_part = tf.clip_by_value(error, 0.0, 1.0)
            linear_part = error - quadratic_part
            loss = tf.reduce_mean(0.5 * tf.square(quadratic_part) + linear_part)

            self.avg_loss += loss.numpy()

        # 오류함수를 줄이는 방향으로 모델 업데이트
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))


# 학습속도를 높이기 위해 흑백화면으로 전처리
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe


if __name__ == "__main__":
    
    file = open('./log.csv', 'w')
    file.write('episode,score,scoreMax,scoreAvg,memoryLength,epsilon,qAvg,avgLoss\n')
    file.close()
    
    # 환경과 DQN 에이전트 생성
    env = gym.make('BreakoutDeterministic-v4')
    agent = DQNAgent(action_size=3)

    global_step = 0
    score_avg = 0
    score_max = 0

    # 불필요한 행동을 없애주기 위한 딕셔너리 선언
    action_dict = {0:1, 1:2, 2:3, 3:3}

    num_episode = 50000
    for e in range(num_episode):
        done = False
        dead = False

        step, score, start_life = 0, 0, 5
        # env 초기화
        observe = env.reset()

        # 랜덤으로 뽑힌 값 만큼의 프레임동안 움직이지 않음
        for _ in range(random.randint(1, agent.no_op_steps)):
            observe, _, _, _ = env.step(1)

        # 프레임을 전처리 한 후 4개의 상태를 쌓아서 입력값으로 사용.
        state = pre_processing(observe)
        history = np.stack((state, state, state, state), axis=2)
        history = np.reshape([history], (1, 84, 84, 4))

        while not done:
            if agent.render:
                env.render()
            global_step += 1
            step += 1

            # 바로 전 history를 입력으로 받아 행동을 선택
            action = agent.get_action(history)
            # 1: 정지, 2: 왼쪽, 3: 오른쪽
            real_action = action_dict[action]

            # 죽었을 때 시작하기 위해 발사 행동을 함
            if dead:
                action, real_action, dead = 0, 1, False

            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, info = env.step(real_action)
            # 각 타임스텝마다 상태 전처리
            next_state = pre_processing(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)

            agent.avg_q_max += np.amax(agent.model(np.float32(history / 255.))[0])

            if start_life > info['ale.lives']:
                dead = True
                start_life = info['ale.lives']

            score += reward
            reward = np.clip(reward, -1., 1.)
            # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장 후 학습
            agent.append_sample(history, action, reward, next_history, dead)

            # 리플레이 메모리 크기가 정해놓은 수치에 도달한 시점부터 모델 학습 시작
            if len(agent.memory) >= agent.train_start:
                agent.train_model()
                # 일정 시간마다 타겟모델을 모델의 가중치로 업데이트
                if global_step % agent.update_target_rate == 0:
                    agent.update_target_model()

            if dead:
                history = np.stack((next_state, next_state,
                                    next_state, next_state), axis=2)
                history = np.reshape([history], (1, 84, 84, 4))
            else:
                history = next_history

            if done:
                # 각 에피소드 당 학습 정보를 기록
                if global_step > agent.train_start:
                    agent.draw_tensorboard(score, step, e)

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                score_max = score if score > score_max else score_max

                log = "episode: {:5d} | ".format(e)
                log += "score: {:4.1f} | ".format(score)
                log += "score max : {:4.1f} | ".format(score_max)
                log += "score avg: {:4.1f} | ".format(score_avg)
                log += "memory length: {:5d} | ".format(len(agent.memory))
                log += "epsilon: {:.3f} | ".format(agent.epsilon)
                log += "q avg : {:3.2f} | ".format(agent.avg_q_max / float(step))
                log += "avg loss : {:3.2f}".format(agent.avg_loss / float(step))
                
                file = open('./log.csv', 'a')
                file.write("{:5d}".format(e) + ',')
                file.write("{:4.1f}".format(score) + ',')
                file.write("{:4.1f}".format(score_max) + ',')
                file.write("{:4.1f}".format(score_avg) + ',')
                file.write("{:5d}".format(len(agent.memory)) + ',')
                file.write("{:.3f}".format(agent.epsilon) + ',')
                file.write("{:3.2f}".format(agent.avg_q_max / float(step)) + ',')
                file.write("{:3.2f}".format(agent.avg_loss / float(step)) + '\n')
                file.close()
                
                print(log)

                agent.avg_q_max, agent.avg_loss = 0, 0

        # 1000 에피소드마다 모델 저장
        if e % 1000 == 0:
            agent.model.save_weights("./save_model/model", save_format="tf")


episode:     0 | score:  1.0 | score max :  1.0 | score avg:  1.0 | memory length:   137 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     1 | score:  2.0 | score max :  2.0 | score avg:  1.1 | memory length:   344 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     2 | score:  1.0 | score max :  2.0 | score avg:  1.1 | memory length:   511 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     3 | score:  0.0 | score max :  2.0 | score avg:  1.0 | memory length:   605 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     4 | score:  1.0 | score max :  2.0 | score avg:  1.0 | memory length:   734 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     5 | score:  1.0 | score max :  2.0 | score avg:  1.0 | memory length:   894 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     6 | score:  0.0 | score max :  2.0 | score avg:  0.9 | memory length:   997 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:     7 | score:  0.

episode:    59 | score:  1.0 | score max :  5.0 | score avg:  1.1 | memory length:  9141 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    60 | score:  0.0 | score max :  5.0 | score avg:  1.0 | memory length:  9244 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    61 | score:  2.0 | score max :  5.0 | score avg:  1.1 | memory length:  9400 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    62 | score:  2.0 | score max :  5.0 | score avg:  1.2 | memory length:  9569 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    63 | score:  0.0 | score max :  5.0 | score avg:  1.1 | memory length:  9670 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    64 | score:  1.0 | score max :  5.0 | score avg:  1.1 | memory length:  9814 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    65 | score:  2.0 | score max :  5.0 | score avg:  1.2 | memory length: 10010 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:    66 | score:  0.

episode:   118 | score:  2.0 | score max :  5.0 | score avg:  1.3 | memory length: 18296 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:   119 | score:  1.0 | score max :  5.0 | score avg:  1.3 | memory length: 18446 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   120 | score:  1.0 | score max :  5.0 | score avg:  1.3 | memory length: 18579 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   121 | score:  2.0 | score max :  5.0 | score avg:  1.3 | memory length: 18775 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   122 | score:  0.0 | score max :  5.0 | score avg:  1.2 | memory length: 18891 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   123 | score:  0.0 | score max :  5.0 | score avg:  1.1 | memory length: 19007 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   124 | score:  1.0 | score max :  5.0 | score avg:  1.1 | memory length: 19145 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   125 | score:  3.

episode:   177 | score:  0.0 | score max :  5.0 | score avg:  0.8 | memory length: 27214 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   178 | score:  0.0 | score max :  5.0 | score avg:  0.7 | memory length: 27311 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   179 | score:  2.0 | score max :  5.0 | score avg:  0.9 | memory length: 27497 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   180 | score:  4.0 | score max :  5.0 | score avg:  1.2 | memory length: 27802 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   181 | score:  3.0 | score max :  5.0 | score avg:  1.4 | memory length: 27999 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:   182 | score:  3.0 | score max :  5.0 | score avg:  1.5 | memory length: 28224 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   183 | score:  1.0 | score max :  5.0 | score avg:  1.5 | memory length: 28390 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   184 | score:  1.

episode:   236 | score:  0.0 | score max :  8.0 | score avg:  0.7 | memory length: 36483 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   237 | score:  0.0 | score max :  8.0 | score avg:  0.7 | memory length: 36585 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   238 | score:  1.0 | score max :  8.0 | score avg:  0.7 | memory length: 36745 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   239 | score:  3.0 | score max :  8.0 | score avg:  0.9 | memory length: 36967 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   240 | score:  1.0 | score max :  8.0 | score avg:  0.9 | memory length: 37113 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   241 | score:  1.0 | score max :  8.0 | score avg:  0.9 | memory length: 37244 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   242 | score:  3.0 | score max :  8.0 | score avg:  1.2 | memory length: 37483 | epsilon: 1.000 | q avg : 0.02 | avg loss : 0.00
episode:   243 | score:  1.

...

episode:  4533 | score: 180.0 | score max : 371.0 | score avg: 113.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.80 | avg loss : 0.01
episode:  4534 | score: 101.0 | score max : 371.0 | score avg: 111.8 | memory length: 100000 | epsilon: 0.020 | q avg : 3.38 | avg loss : 0.01
episode:  4535 | score: 103.0 | score max : 371.0 | score avg: 111.0 | memory length: 100000 | epsilon: 0.020 | q avg : 3.50 | avg loss : 0.00
episode:  4536 | score: 163.0 | score max : 371.0 | score avg: 116.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.62 | avg loss : 0.01
episode:  4537 | score: 290.0 | score max : 371.0 | score avg: 133.6 | memory length: 100000 | epsilon: 0.020 | q avg : 3.95 | avg loss : 0.01
episode:  4538 | score: 60.0 | score max : 371.0 | score avg: 126.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.33 | avg loss : 0.01
episode:  4539 | score: 66.0 | score max : 371.0 | score avg: 120.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.32 | avg loss : 0.01
e

episode:  4591 | score: 28.0 | score max : 384.0 | score avg: 128.0 | memory length: 100000 | epsilon: 0.020 | q avg : 3.22 | avg loss : 0.01
episode:  4592 | score: 141.0 | score max : 384.0 | score avg: 129.3 | memory length: 100000 | epsilon: 0.020 | q avg : 3.61 | avg loss : 0.01
episode:  4593 | score: 95.0 | score max : 384.0 | score avg: 125.9 | memory length: 100000 | epsilon: 0.020 | q avg : 3.50 | avg loss : 0.01
episode:  4594 | score: 227.0 | score max : 384.0 | score avg: 136.0 | memory length: 100000 | epsilon: 0.020 | q avg : 3.89 | avg loss : 0.01
episode:  4595 | score: 83.0 | score max : 384.0 | score avg: 130.7 | memory length: 100000 | epsilon: 0.020 | q avg : 3.49 | avg loss : 0.01
episode:  4596 | score: 38.0 | score max : 384.0 | score avg: 121.4 | memory length: 100000 | epsilon: 0.020 | q avg : 3.30 | avg loss : 0.01
episode:  4597 | score: 69.0 | score max : 384.0 | score avg: 116.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.54 | avg loss : 0.01
epis

episode:  4649 | score: 70.0 | score max : 384.0 | score avg: 155.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.47 | avg loss : 0.01
episode:  4650 | score: 349.0 | score max : 384.0 | score avg: 174.6 | memory length: 100000 | epsilon: 0.020 | q avg : 4.10 | avg loss : 0.01
episode:  4651 | score: 190.0 | score max : 384.0 | score avg: 176.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.85 | avg loss : 0.01
episode:  4652 | score: 58.0 | score max : 384.0 | score avg: 164.3 | memory length: 100000 | epsilon: 0.020 | q avg : 3.60 | avg loss : 0.01
episode:  4653 | score: 62.0 | score max : 384.0 | score avg: 154.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.55 | avg loss : 0.01
episode:  4654 | score: 85.0 | score max : 384.0 | score avg: 147.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.63 | avg loss : 0.01
episode:  4655 | score: 166.0 | score max : 384.0 | score avg: 149.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.89 | avg loss : 0.01
epi

episode:  4707 | score: 28.0 | score max : 384.0 | score avg: 122.9 | memory length: 100000 | epsilon: 0.020 | q avg : 3.44 | avg loss : 0.01
episode:  4708 | score: 205.0 | score max : 384.0 | score avg: 131.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.89 | avg loss : 0.01
episode:  4709 | score: 171.0 | score max : 384.0 | score avg: 135.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.88 | avg loss : 0.01
episode:  4710 | score: 356.0 | score max : 384.0 | score avg: 157.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.43 | avg loss : 0.01
episode:  4711 | score: 102.0 | score max : 384.0 | score avg: 151.7 | memory length: 100000 | epsilon: 0.020 | q avg : 3.73 | avg loss : 0.01
episode:  4712 | score: 238.0 | score max : 384.0 | score avg: 160.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.52 | avg loss : 0.01
episode:  4713 | score: 34.0 | score max : 384.0 | score avg: 147.7 | memory length: 100000 | epsilon: 0.020 | q avg : 3.47 | avg loss : 0.01
e

episode:  4765 | score: 125.0 | score max : 384.0 | score avg: 152.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.87 | avg loss : 0.01
episode:  4766 | score: 283.0 | score max : 384.0 | score avg: 165.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.12 | avg loss : 0.01
episode:  4767 | score: 197.0 | score max : 384.0 | score avg: 168.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.13 | avg loss : 0.01
episode:  4768 | score: 336.0 | score max : 384.0 | score avg: 185.1 | memory length: 100000 | epsilon: 0.020 | q avg : 4.35 | avg loss : 0.01
episode:  4769 | score: 330.0 | score max : 384.0 | score avg: 199.6 | memory length: 100000 | epsilon: 0.020 | q avg : 4.42 | avg loss : 0.01
episode:  4770 | score: 148.0 | score max : 384.0 | score avg: 194.4 | memory length: 100000 | epsilon: 0.020 | q avg : 3.89 | avg loss : 0.01
episode:  4771 | score: 42.0 | score max : 384.0 | score avg: 179.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.52 | avg loss : 0.01
